In [12]:
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator
import plotly.graph_objs as go

In [13]:
filename = "USDC-USD.csv"
df = pd.read_csv(filename)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       731 non-null    object 
 1   Open       731 non-null    float64
 2   High       731 non-null    float64
 3   Low        731 non-null    float64
 4   Close      731 non-null    float64
 5   Adj Close  731 non-null    float64
 6   Volume     731 non-null    int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 40.1+ KB
None


In [14]:
df['Date'] = pd.to_datetime(df['Date'])
df.set_axis(df['Date'], inplace=True)
df.drop(columns=['Open', 'High', 'Low', 'Volume'], inplace=True)

In [15]:
close_data = df['Close'].values
close_data = close_data.reshape((-1,1))

split_percent = 0.80
split = int(split_percent*len(close_data))

close_train = close_data[:split]
close_test = close_data[split:]

date_train = df['Date'][:split]
date_test = df['Date'][split:]

print(len(close_train))
print(len(close_test))

584
147


In [16]:
look_back = 5
train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=20)     
test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=1)


In [21]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(
    LSTM(units = 50,return_sequences = True,
        activation='relu',
        input_shape=(look_back,1))
)
model.add(
    LSTM(50,
        activation='relu',
        input_shape=(look_back,1), return_sequences = True)
)


model.add(LSTM(50, activation = "relu", input_shape=(look_back,1)))

model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

num_epochs = 10
model.fit(train_generator, epochs=num_epochs, verbose=1)

Epoch 1/10
29/29 [==============================] - 3s 6ms/step - loss: 0.8682
Epoch 2/10
29/29 [==============================] - 0s 6ms/step - loss: 0.1965
Epoch 3/10
29/29 [==============================] - 0s 6ms/step - loss: 0.0063
Epoch 4/10
29/29 [==============================] - 0s 6ms/step - loss: 3.2125e-04
Epoch 5/10
29/29 [==============================] - 0s 6ms/step - loss: 3.0585e-05
Epoch 6/10
29/29 [==============================] - 0s 6ms/step - loss: 2.1098e-05
Epoch 7/10
29/29 [==============================] - 0s 6ms/step - loss: 2.1048e-05
Epoch 8/10
29/29 [==============================] - 0s 6ms/step - loss: 1.8743e-05
Epoch 9/10
29/29 [==============================] - 0s 6ms/step - loss: 2.0523e-05
Epoch 10/10
29/29 [==============================] - 0s 5ms/step - loss: 2.0144e-05


In [22]:
prediction = model.predict(test_generator)

close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = prediction.reshape((-1))

trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "Google Stock",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()

In [23]:
close_data = close_data.reshape((-1))
look_back = 5
def predict(num_prediction, model):
    prediction_list = close_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = df['Date'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

num_prediction = 30
forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)


In [24]:
close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = forecast.reshape((-1))

trace1 = go.Scatter(
    x = forecast_dates,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = forecast_dates,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "Google Stock",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace2, trace3], layout=layout)
fig.show()

## ADJUSTED THIS IS FOR PORTFOLIO OPTIMIZATION

In [25]:
adj_close_data = df["Adj Close"].values
adj_close_data = adj_close_data.reshape((-1))
look_back = 15
def predict(num_prediction, model):
    prediction_list = adj_close_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = df['Date'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

num_prediction = 30
adj_forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)



#--------------------------

adj_close_train = adj_close_data[:split]
adj_close_test = adj_close_data[split:]

adj_close_train = adj_close_train.reshape((-1))
adj_close_test = adj_close_test.reshape((-1))
adj_prediction = adj_forecast.reshape((-1))

trace1 = go.Scatter(
    x = forecast_dates,
    y = adj_close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = forecast_dates,
    y = adj_prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = adj_close_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "Google Stock",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[ trace2, trace3], layout=layout)
fig.show()

## ADDING ALL INFO INTO A FILE

In [16]:
newdff = pd.DataFrame(indexes, columns = ["Date"])
newdff["Close"] = newdff["Close"] + forecast

KeyError: 'Close'

In [26]:
#get all close prices values with predicted values (for protfolio prediction)
import datetime

indexes = pd.date_range(start='31/12/2021', end='1/30/2022')

newdf = pd.DataFrame(indexes, columns = ["Date"])
newdf["Close"] = forecast
newdf["Adj Close"] = adj_forecast
dffinal = df.loc[:,["Date" ,"Close","Adj Close"]]
dffinal = dffinal.append(newdf)
dffinal['Date'] = pd.to_datetime(dffinal['Date'])
dffinal

,Date,Close,Adj Close
2020-01-01 00:00:00,2020-01-01,1.004079,1.004079e+00
2020-01-02 00:00:00,2020-01-02,1.005017,1.005017e+00
2020-01-03 00:00:00,2020-01-03,1.005273,1.005273e+00
2020-01-04 00:00:00,2020-01-04,1.009466,1.009466e+00
2020-01-05 00:00:00,2020-01-05,1.008497,1.008497e+00
...,...,...,...
26,2022-01-26,1.000582,1.600657e+13
27,2022-01-27,1.000587,4.304365e+13
28,2022-01-28,1.000592,1.157493e+14
29,2022-01-29,1.000597,3.112339e+14


In [27]:
dffinal.to_csv("USDCpredict.csv", index = False)

## PORTFOLIO OPTIMIZATION STATS

In [ ]:
dffinal = pd.read_csv("ADApredict.csv")

In [ ]:
dffinal.set_axis(dffinal['Date'], inplace=True)
dffinal.drop(columns=["Date"],inplace = True)

In [ ]:
#this are the predicted valyes 

dffinal.tail(60)

In [ ]:
test = pd.read_csv("AOA.csv")

In [ ]:
test